Install Required Libraries

In [3]:
!pip install pyspark numpy


Necessary Libraries

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

Set Up Spark Session

In [5]:

# Initialize Spark session
spark = SparkSession.builder.appName("Big Data Analysis").getOrCreate()


mount the google drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load the Data

In [66]:
# Load the JSON data
df = spark.read.json("/content/drive/My Drive/house_3.json")

# Show the initial schema and data
df.printSchema()
df.show(5)

root
 |-- _corrupt_record: string (nullable = true)
 |-- address: string (nullable = true)
 |-- amenities: string (nullable = true)
 |-- bedroom: string (nullable = true)
 |-- description: string (nullable = true)
 |-- floor: string (nullable = true)
 |-- id: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- poster_id: string (nullable = true)
 |-- poster_type: string (nullable = true)
 |-- price_gel: string (nullable = true)
 |-- price_usd: string (nullable = true)
 |-- product_tree: string (nullable = true)
 |-- room: string (nullable = true)
 |-- space: string (nullable = true)
 |-- time: string (nullable = true)
 |-- title: string (nullable = true)
 |-- views: string (nullable = true)

+---------------+--------------------+--------------------+-------+--------------------+-----+----------+----------+----------+---------+-----------+---------+---------+--------------------+------+---------------+-----------+---------------

define the schema for our JSON data

In [67]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType

schema = StructType([
    StructField("address", StringType(), True),
    StructField("product_tree", StringType(), True),
    StructField("time", StringType(), True),
    StructField("views", IntegerType(), True),
    StructField("title", StringType(), True),
    StructField("id", StringType(), True),
    StructField("price_gel", DoubleType(), True),
    StructField("price_usd", DoubleType(), True),
    StructField("space", DoubleType(), True),
    StructField("room", StringType(), True),
    StructField("bedroom", StringType(), True),
    StructField("floor", IntegerType(), True),
    StructField("description", StringType(), True),
    StructField("amenities", StringType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("poster_type", StringType(), True),
    StructField("poster_id", StringType(), True)
])


 Identify Missing Values and Data Types

In [68]:
from pyspark.sql.functions import isnan, when, count, col

# Show columns with null or NaN values
df.select([count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in df.columns]).show()

# Show data types
df.dtypes

+---------------+-------+---------+-------+-----------+-----+---+--------+---------+---------+-----------+---------+---------+------------+----+-----+----+-----+-----+
|_corrupt_record|address|amenities|bedroom|description|floor| id|latitude|longitude|poster_id|poster_type|price_gel|price_usd|product_tree|room|space|time|title|views|
+---------------+-------+---------+-------+-----------+-----+---+--------+---------+---------+-----------+---------+---------+------------+----+-----+----+-----+-----+
|          41663|      2|        2|      2|       5056|    2|  2|     379|      379|        2|          2|        2|        2|           2|   2|    2|   2|    2|    2|
+---------------+-------+---------+-------+-----------+-----+---+--------+---------+---------+-----------+---------+---------+------------+----+-----+----+-----+-----+



[('_corrupt_record', 'string'),
 ('address', 'string'),
 ('amenities', 'string'),
 ('bedroom', 'string'),
 ('description', 'string'),
 ('floor', 'string'),
 ('id', 'string'),
 ('latitude', 'string'),
 ('longitude', 'string'),
 ('poster_id', 'string'),
 ('poster_type', 'string'),
 ('price_gel', 'string'),
 ('price_usd', 'string'),
 ('product_tree', 'string'),
 ('room', 'string'),
 ('space', 'string'),
 ('time', 'string'),
 ('title', 'string'),
 ('views', 'string')]

 Handle Missing Values and transform data

In [71]:
from pyspark.sql.functions import regexp_replace, split, col, when, trim
from pyspark.sql.types import IntegerType, DoubleType

# Clean and transform the data
df_cleaned = df.withColumn("price_gel", regexp_replace(col("price_gel"), ",", "").cast(DoubleType())) \
    .withColumn("price_usd", regexp_replace(col("price_usd"), ",", "").cast(DoubleType())) \
    .withColumn("space", regexp_replace(regexp_replace(trim(col("space")), "Area:", ""), "[^0-9.]", "").cast(DoubleType())) \
    .withColumn("views", col("views").cast(IntegerType())) \
    .withColumn("floor_info", split(trim(col("floor")), "/")) \
    .withColumn("floor", when(col("floor_info").getItem(0).isNotNull(), col("floor_info").getItem(0).cast(IntegerType())).otherwise(None)) \
    .withColumn("total_floors", when(col("floor_info").getItem(1).isNotNull(), col("floor_info").getItem(1).cast(IntegerType())).otherwise(None)) \
    .drop("floor_info") \
    .withColumn("latitude", col("latitude").cast(DoubleType())) \
    .withColumn("longitude", col("longitude").cast(DoubleType())) \
    .withColumn("amenities", regexp_replace(trim(col("amenities")), ",", "|"))

df_cleaned.describe(["price_gel", "price_usd", "space", "views", "floor", "total_floors", "latitude", "longitude"]).show()
df_cleaned.select([count(when(col(c).isNull(), c)).alias(c) for c in df_cleaned.columns]).show()


# Split product_tree into separate columns
df_cleaned = df_cleaned.withColumn("product_tree_split", split(col("product_tree"), ",")) \
    .withColumn("product_type", col("product_tree_split").getItem(0)) \
    .withColumn("city", col("product_tree_split").getItem(1)) \
    .withColumn("district", col("product_tree_split").getItem(2)) \
    .withColumn("neighborhood", col("product_tree_split").getItem(3)) \
    .drop("product_tree_split")


# Handle missing or invalid values in numeric columns
df_cleaned = df_cleaned.withColumn("space", when(col("space").isNotNull(), col("space")).otherwise(0.0)) \
    .withColumn("views", when(col("views").isNotNull(), col("views")).otherwise(0))

df_cleaned = df_cleaned.drop("_corrupt_record")

# Drop any rows where essential information is missing
df_cleaned = df_cleaned.na.drop(subset=["address", "latitude", "longitude", "product_type", "city", "district"])

# Verify the data cleaning
df_cleaned.show(5)
df_cleaned.describe().show()
df_cleaned.printSchema()
# Check for any remaining missing values
missing_values = df_cleaned.select([col for col in df_cleaned.columns if df_cleaned.filter(df_cleaned[col].isNull()).count() > 0])
missing_values.show()


+-------+------------------+-----------------+------------------+-----------------+-----------------+------------------+-------------------+------------------+
|summary|         price_gel|        price_usd|             space|            views|            floor|      total_floors|           latitude|         longitude|
+-------+------------------+-----------------+------------------+-----------------+-----------------+------------------+-------------------+------------------+
|  count|             41663|            41663|             41663|            41663|            41663|             41663|              41286|             41286|
|   mean|   303180.74454552|92010.22129947436|119.65649353143093| 343.788997431774|6.611837841730072|11.168950867676356|  41.72714931984303|44.496279280450786|
| stddev|2332437.7794166263|707850.4069807667|1601.5270332554157|959.9517539351979|5.064616889399375| 6.326099296077831|0.08925297897968838|0.8919048671315104|
|    min|            3300.0|           1

In [72]:
df_cleaned.describe(["price_gel", "price_usd", "space", "views", "floor", "total_floors", "latitude", "longitude"]).show()
df_cleaned.select([count(when(col(c).isNull(), c)).alias(c) for c in df_cleaned.columns]).show()


+-------+-----------------+-----------------+------------------+-----------------+-----------------+------------------+-------------------+------------------+
|summary|        price_gel|        price_usd|             space|            views|            floor|      total_floors|           latitude|         longitude|
+-------+-----------------+-----------------+------------------+-----------------+-----------------+------------------+-------------------+------------------+
|  count|            41286|            41286|             41286|            41286|            41286|             41286|              41286|             41286|
|   mean| 303351.142275832| 92061.9403671947|119.91931768638302|337.6937460640411| 6.62539359589207|11.191566148331153|  41.72714931984303|44.496279280450786|
| stddev|2342846.660707693|711009.3025026474|1608.8103784888133|954.0329434338029|5.070424082057078| 6.331992092106217|0.08925297897968838|0.8919048671315104|
|    min|           3300.0|           1000.0| 

Exploratory Data Analysis

In [73]:
# Distribution of price in GEL and USD
df_cleaned.select("price_gel").describe().show()
df_cleaned.select("price_usd").describe().show()

# Distribution of space and views
df_cleaned.select("space").describe().show()
df_cleaned.select("views").describe().show()

# Count the occurrences of each unique value in categorical variables
df_cleaned.groupBy("bedroom").count().show()
df_cleaned.groupBy("amenities").count().show()


+-------+-----------------+
|summary|        price_gel|
+-------+-----------------+
|  count|            41286|
|   mean| 303351.142275832|
| stddev|2342846.660707693|
|    min|           3300.0|
|    max|         3.2951E8|
+-------+-----------------+

+-------+-----------------+
|summary|        price_usd|
+-------+-----------------+
|  count|            41286|
|   mean| 92061.9403671947|
| stddev|711009.3025026474|
|    min|           1000.0|
|    max|            1.0E8|
+-------+-----------------+

+-------+------------------+
|summary|             space|
+-------+------------------+
|  count|             41286|
|   mean|119.91931768638302|
| stddev|1608.8103784888133|
|    min|               1.0|
|    max|          133000.0|
+-------+------------------+

+-------+-----------------+
|summary|            views|
+-------+-----------------+
|  count|            41286|
|   mean|337.6937460640411|
| stddev|954.0329434338029|
|    min|                0|
|    max|            80933|
+-------

Analyze relationships between pairs of variables

In [75]:
# Relationship between price and space
df_cleaned.select("price_gel", "space").groupBy("space").avg("price_gel").orderBy("space").show()

# Relationship between views and price
df_cleaned.select("views", "price_gel").groupBy("views").avg("price_gel").orderBy("views").show()

# Relationship between space, city, and price
df_cleaned.groupBy("city", "space").avg("price_gel").orderBy("city", "space").show()

# Analyze the impact of amenities on price
df_cleaned.groupBy("amenities").avg("price_gel").orderBy("avg(price_gel)", ascending=False).show()

# Analyze average price by city
df_cleaned.groupBy("city").avg("price_gel").orderBy("avg(price_gel)", ascending=False).show()


+-----+------------------+
|space|    avg(price_gel)|
+-----+------------------+
|  1.0|119133.33333333333|
| 6.25|          280100.0|
|  7.0|           21400.0|
|  9.0|           23100.0|
| 10.0|           26350.0|
|10.72|           42800.0|
| 12.0|           58200.0|
| 13.0|           45350.0|
| 14.0|45033.333333333336|
| 15.0| 48485.71428571428|
| 16.0| 51323.07692307692|
| 17.0|           47800.0|
| 17.5|           89000.0|
| 18.0| 68569.69696969698|
| 19.0|           76375.0|
|19.56|           39500.0|
| 20.0|           53780.0|
| 21.0| 55978.57142857143|
| 21.5|           65550.0|
| 22.0|60769.565217391304|
+-----+------------------+
only showing top 20 rows

+-----+------------------+
|views|    avg(price_gel)|
+-----+------------------+
|    0| 275243.2098765432|
|    1|210633.33333333334|
|    2| 248234.1463414634|
|    3|199995.45454545456|
|    4|173933.33333333334|
|    5| 691861.1111111111|
|    6|          372016.0|
|    7| 288408.1967213115|
|    8| 298144.3396226415|
| 